In [ ]:
# common import for all cells
SEED=202
DATA_DIR='/root/share/project/udacity/project2_02/data'
OUT_DIR ='/root/share/out/udacity/08'

# standard libs
import pickle
import csv
from timeit import default_timer as timer
import os
import sys


#visualisation
%matplotlib inline

import matplotlib.pyplot as plt 
from IPython.display import Image
from IPython.display import display

# numerical libs 
import cv2
import math

import random
import numpy as np
random.seed(SEED)
np.random.seed(SEED)

import tensorflow as tf
tf.set_random_seed(SEED)

from tensorflow.python.training import moving_averages
from tensorflow.contrib.framework import add_model_variable
sess = tf.InteractiveSession()
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data
data_dir = '/Users/shleifer/flow/CarND-Traffic-Sign-Classifier-Project/traffic-signs-data/'
DATA_DIR = data_dir
training_file = data_dir + 'train.p'
validation_file= data_dir + 'valid.p'
testing_file =data_dir + 'test.p'

#testing_file = ?

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
#pd.Series(y_test)

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
classnames = pd.read_csv('signnames.csv', index_col=0).SignName

signnames_path =  'udacity-driverless-car-nd-p2/data/signnames_all.jpg'


%matplotlib inline

import numpy as np

from matplotlib.pyplot import imshow

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

X_train_normalized = X_train/127.5-1
X_valid_normalized = X_valid/127.5-1
X_test_normalized = X_test/127.5-1

name_values = np.genfromtxt('signnames.csv', skip_header=1, dtype=[('myint','i8'), ('mysring','S55')], delimiter=',')

# TODO: Number of training examples 
num_train = len(X_train)

# TODO: Number of testing examples.
num_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
_, height, width, channel = X_train.shape
image_shape = (height, width, channel)

# TODO: How many unique classes/labels there are in the dataset.
num_class = len(np.unique(y_train))

### Model Architecture

### Train, Validate and Test the Model

In [4]:
n_classes=43
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='x')
y = tf.placeholder(tf.uint8, shape=(None), name='y')
one_hot_y = tf.one_hot(y, n_classes)
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
IS_TRAIN_PHASE = tf.placeholder(tf.bool, name='IS_TRAIN_PHASE')

In [5]:
from tensorflow.contrib.layers import flatten

mu = 0
sigma = 0.1


def conv2d_relu(x, w_shape, strides=1, padding='VALID'):
    W = tf.Variable(tf.truncated_normal(shape=w_shape, mean=mu, stddev=sigma))
    b = tf.Variable(tf.zeros(w_shape[-1]))
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1],
                     padding=padding)
    x = tf.nn.bias_add(x, b)

    return tf.nn.relu(x)


def fc_layer(x, num_outputs):
    fc1 = tf.contrib.layers.fully_connected(x, num_outputs, activation_fn=tf.nn.relu)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    return fc1



def LeNet(x, n_channels=3):
    conv1 = conv2d_relu(x, (5, 5, 3, 6))
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    conv2 = conv2d_relu(conv1, (5, 5, 6, 16))
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv2 = tf.contrib.layers.batch_norm(conv2)
    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0 = flatten(conv2)
    fc1 = fc_layer(fc0, 120)
    fc2 = fc_layer(fc1, 84)
    logits = tf.contrib.layers.fully_connected(fc2, n_classes)
    return logits

In [6]:
rate = 0.001

with tf.name_scope('metrics') as scope:
    logits = LeNet(x)
    tf.summary.histogram('logits', logits)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, 
                                                            logits=logits)
    
    loss_operation = tf.reduce_mean(cross_entropy)
    #tf.summary.scalar('loss_operation', loss_operation)
    optimizer = tf.train.AdamOptimizer(learning_rate = rate)
    training_operation = optimizer.minimize(loss_operation)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
    tf.summary.scalar('loss', loss_operation)
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),)
    tf.summary.scalar('accuracy', accuracy_operation)

In [7]:

X_train= X_train_normalized
X_valid = X_valid_normalized

In [8]:
assert np.round(X_valid.std(), 2) == .55, 'DATA NOT NORMALIZED'

In [9]:
assert np.round(X_train.std(), 2) == .53, 'DATA NOT NORMALIZED'

In [10]:

saver = tf.train.Saver()

def evaluate(X_data, y_data, num_examples=None):
    if num_examples is None:
        num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, 
                            feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: 1., IS_TRAIN_PHASE: True})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


In [11]:
summ_dir = '/tmp/tensorflow/'
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summ_dir + '/train',
                                      sess.graph)
test_writer = tf.summary.FileWriter(summ_dir + '/test')

In [12]:
EPOCHS = 20
BATCH_SIZE = 156
from sklearn.utils import shuffle


In [13]:
def log_accuracy(verbose=False):
    validation_accuracy = evaluate(X_valid, y_valid)
    train_accuracy = evaluate(X_train, y_train)
    if verbose:
        print("Train={:.3f}, Validation={:.3f}".format(
            train_accuracy, validation_accuracy))
    return train_accuracy, validation_accuracy

In [14]:
y_valid

array([41, 41, 41, ..., 25, 25, 25], dtype=uint8)

In [17]:
%%time

train_accs = {}
valid_accs = {}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    print("Training...")
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for j, offset in enumerate(range(0, num_examples, BATCH_SIZE)):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            
            sess.run(training_operation, 
                     feed_dict={x: batch_x, y: batch_y,
                               keep_prob: .5, IS_TRAIN_PHASE: True})
            if j % 100 == 0:  # Record summaries and test-set accuracy
                summary = sess.run(merged,
                                        feed_dict={x: X_valid,
                                                   y: y_valid,
                                                   keep_prob: 1.,
                                                  IS_TRAIN_PHASE: False})
                test_writer.add_summary(summary, i)
               
           

        print("EPOCH {} ...".format(i+1))
        
        a,b = log_accuracy(verbose=True)
        train_accs[i] = a
        valid_accs[i] = b
        
    saver.save(sess, './lenet_dropout_refactor_bn')
    print("Model saved")

Training...
EPOCH 1 ...
Train=0.666, Validation=0.562
EPOCH 2 ...
Train=0.846, Validation=0.757
EPOCH 3 ...
Train=0.905, Validation=0.824
EPOCH 4 ...
Train=0.942, Validation=0.880
EPOCH 5 ...
Train=0.962, Validation=0.902
EPOCH 6 ...
Train=0.972, Validation=0.919
EPOCH 7 ...
Train=0.975, Validation=0.918
EPOCH 8 ...
Train=0.978, Validation=0.926
EPOCH 9 ...
Train=0.982, Validation=0.934
EPOCH 10 ...
Train=0.983, Validation=0.935
EPOCH 11 ...
Train=0.984, Validation=0.935
EPOCH 12 ...
Train=0.985, Validation=0.937
EPOCH 13 ...
Train=0.985, Validation=0.937
EPOCH 14 ...
Train=0.985, Validation=0.931
EPOCH 15 ...
Train=0.985, Validation=0.932
EPOCH 16 ...
Train=0.987, Validation=0.943
EPOCH 17 ...
Train=0.987, Validation=0.932
EPOCH 18 ...
Train=0.987, Validation=0.938
EPOCH 19 ...
Train=0.987, Validation=0.938
EPOCH 20 ...
Train=0.987, Validation=0.941
Model saved
CPU times: user 35min 43s, sys: 7min 18s, total: 43min 2s
Wall time: 10min 59s


In [ ]:
import sys
sys.path= ['.'] + sys.path
from utils.lenet_udacity import get_n_params
%load_ext autoreload
%autoreload 2

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './lenet_dropout_refactor')
    params = get_n_params()

In [ ]:
sess = tf.Session()

In [ ]:
saver.restore(sess, './lenet_dropout_refactor')

In [ ]:
v = tf.trainable_variables()

In [ ]:
v[0].get_shape()

In [ ]:
shapes = {i: x.get_shape() for i, x in enumerate(v)}

In [ ]:
shapes

In [18]:
with tf.Session() as sess:
    saver.restore(sess, './lenet_dropout_refactor_bn')
    validation_accuracy = evaluate(X_valid, y_valid)
    train_accuracy = evaluate(X_train, y_train)
    test_accuracy = evaluate(X_test_normalized, y_test)

In [19]:
train_accuracy, validation_accuracy, test_accuracy

(0.98729848006651411, 0.94104308765761702, 0.94085510105531744)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
!mv /Users/shleifer/Downloads/traffic\ signs .

In [ ]:
mv traffic\ signs real_signs

In [ ]:
import glob
files = glob.glob('/Users/shleifer/Downloads/TrafficSignsRuDe/Germany/*jpg')

In [ ]:
import cv2 

import os
import matplotlib.image as mpimg
i = 0



In [ ]:
%history -g cv2.re

In [ ]:
%history -g cv2.resize

In [ ]:
imgs[0].shape

In [ ]:
imgs[3].std()

In [ ]:
np.array([(img / 127.5 - 1)]).shape

In [ ]:
sess = tf.Session()

In [ ]:
saver.restore(sess, './lenet_dropout')

In [ ]:
classnames

In [ ]:

for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, (32,32))
    im_normalized = np.array([(img / 127.5 - 1)])
    softmax_prob = sess.run(tf.nn.softmax(logits),
                                feed_dict={x:im_normalized, 
                                           keep_prob:1.0})

    pred_name = classnames.loc[pd.Series(softmax_prob[0]).idxmax()]
    fig = plt.figure(figsize=(15,2))
    ax = fig.add_subplot(1,2,1) 
    img = mpimg.imread(file)
    ax.imshow(img)

    ax = fig.add_subplot(1,2,2) 
    ax.set_xticks(np.arange(0,43,1))
    plt.plot(softmax_prob[0])
    plt.title('Highest: {}: {:.2f}'.format(pred_name, max(softmax_prob[0])))
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize('x-small') 
        tick.label.set_rotation('vertical')
    plt.grid()
    plt.show()

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")